In [1]:
import numpy as np

def init_proposal(sampler, params):
    return lambda n: sampler(**params, size=n)

def CSMC(obs, N, conditional_traj, proposals_samplers, proposals_dens, transition_dens, obs_dens, init_dens):
    particles = np.array([[]])
    weights = [[]]
    particles[0] = np.append(proposals_samplers[0](N-1), conditional_traj[0][np.newaxis,:])
    for i in range(N):
        weights[0].append(obs_dens[0](obs[0], particles[0, i]) * init_dens(particles[0, i]) / proposals_dens[0](particles[0, i]))
    T = len(obs)
    ancestors = np.array([[]])
    for t in range(1, T):
        ancestors[t-1] = np.append(np.random.choice(range(1, N+1), size=N-1, p=weights[t-1]), N)
        
        new_particles = []
        for i in range(N-1):
            new_particles.append(proposals_samplers[0](1, particles[0:t-1, ancestors[t-1,i]]))
        new_particles.append(conditional_traj[t])
        particles[t] = np.array(new_particles)
        weights.append([])
        for i in range(N):
            particles[0:t, i] = particles[0:t, ancestors[t-1,i]]
            weights[t].append(obs_dens[t](obs[t], particles[0:t+1, i]) * transition_dens[t-1](particles[0:t, ancestors[t-1,i]]) / proposals_dens[t](particles[0:t, ancestors[t-1,i]]))
        


In [63]:
import os
os.chdir('/Users/corentin/Documents/M2/BIPODS/Project/iPMCMC/ipmcmc')

In [64]:
from scipy.spatial.transform import Rotation as R
%run generate_data.py
%run utils.py

np.random.seed(420)

# Parameters
r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)

# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma, transition_coeffs=alpha, observation_coeffs=beta)

In [60]:
l_observations.shape

(50, 20)

In [65]:
transition_dens = np.array([gaussian_density(mu, start_var)] + [gaussian_density_var(omega, alpha)] * (t_max-1))
proposals_dens = np.copy(transition_dens)
obs_dens = np.array([gaussian_density_var(sigma, beta)] * t_max)

proposals_samplers = np.array([lambda n: np.random.multivariate_normal(mu, start_var, size=n)] + [lambda mu: np.random.multivariate_normal(mu, omega)] * (t_max - 1))

conditional_traj = l_states
N = 32

In [51]:
transition_dens[0]


<function gaussian_density.<locals>.dens_func(x)>

In [67]:
%run csmc.py

particles, weights = CSMC(l_observations, N, conditional_traj, proposals_samplers, proposals_dens, transition_dens, obs_dens)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

[[-0.50170531  1.04068852  1.62698387]
 [ 0.33384116  0.99340188  1.1345917 ]
 [ 0.17148658  0.93744378  1.62101318]
 [ 0.36847673  0.36561076  0.81734616]
 [-0.10589992  1.08578217  0.74194158]
 [-0.24699728  1.43413504  0.89132101]
 [-0.20117617  1.24179453  0.74947656]
 [-0.19071568  1.54448572  0.27930383]
 [-0.11032444  1.34820326  1.4599226 ]
 [ 0.15274258  0.70012542  0.80941941]
 [-0.10950861  1.30738528  0.39357352]
 [ 0.01498976  0.69287057  1.28668052]
 [ 0.24785446  1.3736419   0.8927272 ]
 [-0.20887344  1.03612004  0.65600623]
 [-0.36910357  0.74011164  0.75600202]
 [-0.13489697  1.28852297  0.60432013]
 [ 0.16723946  0.81533776  0.81079233]
 [-0.28643024  0.82809063  0.86769238]
 [-0.31276472  0.87842174  0.88587284]
 [ 0.12761458  1.15714584  0.46966809]
 [-0.02252211  0.47466515  0.27208194]
 [ 0.13336899  2.04941201  1.32863193]
 [-0.4088136   2.11271922  1.08119452]
 [-0.23800956  1.21126769  0.97695511]
 [-0.2211363   1.12196548  1.24784228]
 [-0.07346358  0.72323759

In [62]:
l_observations.shape

(50, 20)

True